Here, I will test the fine-tuned model.

In [ ]:
from matplotlib import pyplot as plt
from torch import nn, optim
from torchvision import datasets, models, transforms

import copy
import os
import time
import torch
import torchvision

print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)

torch.set_printoptions(precision=3, sci_mode=False)

In [ ]:
data_dir = 'D:/my_gestures'

model_name = '02_MobileNet_V3_Small_Weights'

classes = ['fist', 'one', 'two', 'three', 'four', 'five']
num_classes = len(classes)

batch_size = 1

num_epochs = 1

feature_extract = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model = torch.load("./saved_models/02_MobileNet_V3_Small_Weights.pth")

model = model.to(device)

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

In [ ]:
img = next(iter(dataloaders_dict['val']))
disp = img[0].permute(0, 2, 3, 1)
plt.imshow(disp[0])

model.eval()
with torch.no_grad():
    tensor = img[0].to(device)
    start = time.perf_counter()
    pred = model(tensor)
    end = (time.perf_counter() - start) * 1000
    print(f"{end:.2f}ms")
    print(pred)
    predicted = classes[pred[0].argmax(0)]
    sm = torch.nn.Softmax(1)(pred)
    print(sm)
    confidence = sm[0][pred[0].argmax(0)]
    print(f'predicted: {predicted}, confidence: {confidence * 100:.2f} %')